In [ ]:
import torch
# from utils import train_transform,val_transform,imagedataloader
from utils import train_transform, imagedataloader
from RetNet import *
from train_retnet import Trainer
import torch.optim as optim
import torch.nn as nn
from Nets import visiontransformer, altgvt, mobilenetv3
testname = "statefarmMB_1_2"

MobileV3配置和GVT相同

1_1 ,2.0G - 0.8G 67min

1_2,2.4G -0.8G 67min

1_3 2.0G -0.8G
1_4 256min 2.4-0.8G

ALTGVT 配置见config Batchsize = 128, 显存2.2-0.8=1.410步1e-3准确率89%, 12min

GVT1_1 3.3-0.8G, B256, EP50[5,15,30],61min

GVT1_2 5.3-0.8G, B512, EP50[5,15,30],63min

GVT1_3 3.2-0.8G, B256 EP200[20,50,100,150], 246min

GVT1_4 3.2-0.8G, B256 EP200[20,50,100,150], 246min

In [ ]:
# statefarmMSR4_1 B = 256, 157min,100epoch
# 5_1,bs=64,imgs=256,50[5,15,30],317min
# 6_1,bs=256,预处理和原论文一样(变简单了),imgs=64
# 7-1
def main():

    # Training parameters
    split = (0.7, 0.3)
    batch_size = 512  # 在256测试了16, 32, 64, 128， 64最快(有chunk)
    epochs = 50
    lr = 1e-3
    device = torch.device("cuda")
    save_model_every_n_epochs = 20
    trainloader, valloader, _ = imagedataloader(split=split,
                                                path="./data/statefarm/", batchsize=batch_size, transform=train_transform,
                                                )
    # Create the model, optimizer, loss function and trainer
    # [ ]
    # model = visiontransformer.to(device)
    # [ ]
    # model = altgvt.to(device)
    # [ ]
    # model = mobilenetv3.to(device)
    model = ImageRetResNet(device=device).to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5, 15, 30], gamma=0.1)
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
    trainer = Trainer(model, optimizer, loss_fn, testname, device=device, scheduler=scheduler)
    trainer.train(trainloader, valloader, epochs, save_model_every_n_epochs=save_model_every_n_epochs)

In [ ]:
main()

In [ ]:
import matplotlib.pyplot as plt
import os
import json
testname = "statefarmMSR6_1"
metrics_file = os.path.join(f"./experiments/{testname}/", 'metrics.json')
with open(metrics_file, 'r') as f:
    data = json.load(f)
    train_losses = data['train_losses']
    test_losses = data['test_losses']
    accuracies = data['accuracies']

# Create two subplots of train/test losses and accuracies
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
ax1.plot(train_losses, label="Train loss")
ax1.plot(test_losses, label="Val loss")
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.legend()
ax2.plot(accuracies)
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy")
plt.savefig(f"{testname}.svg", format="svg")
plt.show()

In [ ]:
# from torchstat import stat
# model = altgvt
# stat(model, (3, 64, 64))

In [ ]:
# a = torch.randn(3, 64, 64)
# a = a.unsqueeze(0)
# print(a.shape)

**3,64,64**

Total params: 1,253,446
Total memory: 1.06MB
Total MAdd: 19.98MMAdd
Total Flops: 10.1MFlops
Total MemR+W: 6.48MB

---

**3,256,256**

Total params: 1,253,446
Total memory: 16.91MB
Total MAdd: 319.47MMAdd
Total Flops: 161.44MFlops
Total MemR+W: 32.03MB

In [ ]:
from torchsummary import summary
model = visiontransformer
summary(model, input_size=(3, 64, 64), device="cpu")

Total params: 1,256,326
Trainable params: 1,256,326
Non-trainable params: 0
Input size (MB): 0.05
Forward/backward pass size (MB): 2.63
Params size (MB): 4.79
Estimated Total Size (MB): 7.47